Installing Python-Terrier 


In [ ]:
!pip install python-terrier

import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 38.7 MB/s 
     |████████████████████████████████| 69 kB 5.2 MB/s 
     |████████████████████████████████| 303 kB 49.4 MB/s 
     |████████████████████████████████| 46 kB 3.1 MB/s 
     |████████████████████████████████| 45 kB 1.7 MB/s 
     |████████████████████████████████| 1.2 MB 35.4 MB/s 
     |████████████████████████████████| 294 kB 40.4 MB/s 
     |████████████████████████████████| 126 kB 44.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 285 kB 49.4 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.1-py3-none-any.whl size=104091 sha256=d1a2f8bd2e27cc1601f9ccfedeb34b4f8a0e474c388d7

In [ ]:
import pyterrier as pt
pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



Connecting Google Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
name_dir=("/content/drive/MyDrive/IRDM")

Mounted at /content/drive


Fetching Files in the directory to Index 

In [ ]:
xml_list=[]
for root, folder, data in os.walk(name_dir):
    for i in data:
        if ".xml" in i:
          xml_list.append((os.path.join(root,i)))

Indexing the Files 

In [ ]:
indexer = pt.TRECCollectionIndexer("/content/drive/MyDrive/IRDM/xml_dir/ind_3",blocks=True,meta={'docno':50,'text': 2048,})

indexref = indexer.index(xml_list)

Reading the Index 


In [ ]:
index_ref = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/ind_3/data.properties")
index= pt.IndexFactory.of(index_ref)

In [ ]:
print(index.getCollectionStatistics())

Number of documents: 3593829
Number of terms: 209849
Number of postings: 324936225
Number of fields: 0
Number of tokens: 500325872
Field names: []
Positions:   true



Baselines (BM25 and DPH)

In [ ]:
id_template = 'spotify:episode:{}_{}'
def get_ep(a,b):
  return id_template.format(a,b) 

Tuning the BM25 Model

In [ ]:
b=[0.30,0.60,0.75,0.90]
k =[0.5,1.2,1.5,1.6,2,2.5,3]

max=0
b_opt=0
k_opt=0
for i in range(len(b)):
  for j in range(len(k)):
    topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
    bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":b[i]}, metadata=["docno",'text'],properties={"bm25.k_1":k[j]})
    res=bm25.transform(topics)
    df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
    df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
    df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
    df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
    df["docno"]=df["docno"]+".0"
    rsl=df[["qid","docno","rank","score"]]
    qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
    qrels = pt.io.read_qrels(qrels_path)
    a=pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])
    util=a["ndcg_cut_30"]
    if util > max: 
      max=util
      b_opt=b[i]
      k_opt=k[j]

print("The optimal values are")
print("B is {}".format(b_opt))
print("K1 is {}".format(k_opt))
        

The optimal values are
B is 0.6
K1 is 0.5


In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


res=bm25.transform(topics)
res_2=ret.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])

def get_ep(a,b):
  return id_template.format(a,b) 
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)

print("BM 25")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("DPH")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25
{'map': 0.2250640250319111, 'ndcg': 0.4144940458278415, 'ndcg_cut_30': 0.27352479286700465, 'P_10': 0.325}
DPH
{'map': 0.22365605208003647, 'ndcg': 0.41517404363420496, 'ndcg_cut_30': 0.270274529619579, 'P_10': 0.325}


Tuning PL2 Model 






In [ ]:
c=[0.1, 1, 5, 10, 20, 100]

max=0
c_opt=0
for i in range(len(c)):
    topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
    pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":c[i]},metadata=["docno",'text'])
    res=pl2.transform(topics)
    df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
    df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
    df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
    df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
    df["docno"]=df["docno"]+".0"
    rsl=df[["qid","docno","rank","score"]]
    qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
    qrels = pt.io.read_qrels(qrels_path)
    a=pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])
    util=a["ndcg_cut_30"]
    if util > max: 
      max=util
      c_opt=c[i]


print("The optimal values are")
print("c is {}".format(c_opt))
        

The optimal values are
c is 20


In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
dir = pt.BatchRetrieve(index, controls={"wmodel":"DirichletLM"}, metadata=["docno",'text'])
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])



import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=dir.transform(topics)
res_2=pl2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"

rsl=df[["qid","docno","rank","score"]]

rsl_2=df_2[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)

print("Dirichlet LM")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("PL 2 ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

Dirichlet LM
{'map': 0.2055113781274805, 'ndcg': 0.4019538896371915, 'ndcg_cut_30': 0.2553287975693738, 'P_10': 0.35}
PL 2 
{'map': 0.22161891651584797, 'ndcg': 0.4259666693266723, 'ndcg_cut_30': 0.2820225728886583, 'P_10': 0.36250000000000004}


In [ ]:

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})

bm25 = get_retr(index, 0.6, 0.5) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
combined = bm25 + dph


qe = pt.BatchRetrieve(index, controls={'wmodel': "DPH"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=10, fb_docs=3) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=combined.transform(topics)
res_2=qe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + DPH")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("DPH QE ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

00:51:08.848 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
BM 25 + DPH
{'map': 0.2247124833049004, 'ndcg': 0.4160686973395136, 'ndcg_cut_30': 0.2642382004792583, 'P_10': 0.325}
DPH QE 
{'map': 0.23151433194054494, 'ndcg': 0.38511529914024545, 'ndcg_cut_30': 0.2649001186910151, 'P_10': 0.32499999999999996}


Intersection and Union of Scores 

In [ ]:

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})

bm25 = get_retr(index, 0.6, 0.5) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
union = bm25 | dph

intersection = bm25 & dph


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=union.transform(topics)
res_2=intersection.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
print("BM 25  DPH Union")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("BM 25 Intersection ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25  DPH Union
{'map': 0.02426358023517412, 'ndcg': 0.2376214568282763, 'ndcg_cut_30': 0.020071638358417708, 'P_10': 0.025}
BM 25 Intersection 
{'map': 0.026435408289413844, 'ndcg': 0.23913957499440827, 'ndcg_cut_30': 0.020071638358417708, 'P_10': 0.025}


In [ ]:
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")

mod= bm25+pl2
mod2= pl2+dph
import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=mod.transform(topics)
res_2=mod2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + PL2")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("pl2 + DPH ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25 + PL2
{'map': 0.2223147452564862, 'ndcg': 0.41737350497107806, 'ndcg_cut_30': 0.2750442993137609, 'P_10': 0.3375}
pl2 + DPH 
{'map': 0.22275188312092867, 'ndcg': 0.426396182085419, 'ndcg_cut_30': 0.2817705181847363, 'P_10': 0.36250000000000004}


Evaluation on the Test Set 


In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


res=bm25.transform(topics)
res_2=ret.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])

def get_ep(a,b):
  return id_template.format(a,b) 
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)

print("BM 25")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("DPH")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25
{'map': 0.3202754057459926, 'ndcg': 0.5333137317273846, 'ndcg_cut_30': 0.376825468706693, 'P_10': 0.4729166666666667}
DPH
{'map': 0.3147094900432008, 'ndcg': 0.5306140063889719, 'ndcg_cut_30': 0.37211042436513303, 'P_10': 0.46041666666666664}


In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
dir = pt.BatchRetrieve(index, controls={"wmodel":"DirichletLM"}, metadata=["docno",'text'])
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])



import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=dir.transform(topics)
res_2=pl2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"

rsl=df[["qid","docno","rank","score"]]

rsl_2=df_2[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)

print("Dirichlet LM")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("PL 2 ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

Dirichlet LM
{'map': 0.3032109048163873, 'ndcg': 0.5205734106300994, 'ndcg_cut_30': 0.36034235871050674, 'P_10': 0.4520833333333334}
PL 2 
{'map': 0.3203301240579921, 'ndcg': 0.5369086594226401, 'ndcg_cut_30': 0.3856113515122643, 'P_10': 0.47916666666666674}


In [ ]:

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})

bm25 = get_retr(index, 0.6, 0.5) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
combined = bm25 + dph


qe = pt.BatchRetrieve(index, controls={'wmodel': "DPH"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=10, fb_docs=3) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=combined.transform(topics)
res_2=qe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + DPH")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("DPH QE ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

02:14:06.784 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
BM 25 + DPH
{'map': 0.31948797338163054, 'ndcg': 0.5350740298395601, 'ndcg_cut_30': 0.3755865400510014, 'P_10': 0.4583333333333334}
DPH QE 
{'map': 0.3322469354019687, 'ndcg': 0.5425089432305128, 'ndcg_cut_30': 0.39431028666872137, 'P_10': 0.4708333333333334}


In [ ]:

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})

bm25 = get_retr(index, 0.6, 0.5) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
union = bm25 | dph

intersection = bm25 & dph


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=union.transform(topics)
res_2=intersection.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25  DPH Union")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("BM 25 Intersection ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25  DPH Union
{'map': 0.046303656210762066, 'ndcg': 0.2990217862718269, 'ndcg_cut_30': 0.03763803837133194, 'P_10': 0.029166666666666674}
BM 25 Intersection 
{'map': 0.05915750569942802, 'ndcg': 0.30936610658092584, 'ndcg_cut_30': 0.04839162462111141, 'P_10': 0.04375000000000001}


In [ ]:
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")

mod= bm25+pl2
mod2= pl2+dph
import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=mod.transform(topics)
res_2=mod2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + PL2")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("pl2 + DPH ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25 + PL2
{'map': 0.32169708284705717, 'ndcg': 0.5391265662070295, 'ndcg_cut_30': 0.3851847555337686, 'P_10': 0.48333333333333345}
pl2 + DPH 
{'map': 0.31834581358744846, 'ndcg': 0.5382053910649885, 'ndcg_cut_30': 0.37865648854345046, 'P_10': 0.4645833333333334}


*More On Query Expansion*


DPH with 15 Terms 


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "DPH"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=3) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

19:36:35.640 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.27436183054573854, 'ndcg': 0.4485519984379497, 'ndcg_cut_30': 0.30666349424971995, 'P_10': 0.3875}


BM25 as the first ranker (with 10 terms)

In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=10, fb_docs=3) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

19:38:59.581 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.27344076274051743, 'ndcg': 0.4487360096635296, 'ndcg_cut_30': 0.30855921591131485, 'P_10': 0.3875}


BM 25 as the final ranker with 15 terms 


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=3) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

19:40:30.092 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.27436183054573854, 'ndcg': 0.4485519984379497, 'ndcg_cut_30': 0.30666349424971995, 'P_10': 0.3875}


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=10, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:12:10.914 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.29104825293081976, 'ndcg': 0.44972399030108157, 'ndcg_cut_30': 0.3188111792728916, 'P_10': 0.3875}


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:14:06.667 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.2912846570469991, 'ndcg': 0.45041761499567534, 'ndcg_cut_30': 0.3154571997207201, 'P_10': 0.4}


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=20, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:14:47.830 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.29416775574078113, 'ndcg': 0.45167378839993744, 'ndcg_cut_30': 0.31174774087523094, 'P_10': 0.41250000000000003}


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "PL2","c":20})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:18:15.005 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.2930636823402478, 'ndcg': 0.4530442261142845, 'ndcg_cut_30': 0.3183466661379537, 'P_10': 0.4}


Evaluation of Query Expansion Models on Training set


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "DPH"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=3) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:22:05.715 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.33277637807064436, 'ndcg': 0.5425046256940795, 'ndcg_cut_30': 0.3969099795512426, 'P_10': 0.4604166666666667}


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=3) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:24:24.580 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.33919971900977575, 'ndcg': 0.5459540156285886, 'ndcg_cut_30': 0.39949139817791207, 'P_10': 0.48750000000000004}


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

15:30:17.138 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.33079485859673624, 'ndcg': 0.5373100931338427, 'ndcg_cut_30': 0.38830125301364155, 'P_10': 0.46875}


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=20, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:41:41.601 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.3461191976904771, 'ndcg': 0.5520133109315944, 'ndcg_cut_30': 0.4101181714572662, 'P_10': 0.4750000000000001}


In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "PL2","c":20})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:28:29.803 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.3499531864018813, 'ndcg': 0.557134573263066, 'ndcg_cut_30': 0.40925973546515787, 'P_10': 0.48125}


Learning to Rank Approaches 

In [ ]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.1,
      gamma=0.6,
      min_child_weight=0.20,
      max_depth=6,
      verbose=2,
      random_state=42)

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2","WMODEL:DPH",])
lmart_x_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels_val = pt.io.read_qrels(qrels_path)
val = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
lmart_x_pipe.fit(train.head(5), qrels_train,train.tail(3),qrels_train)

 
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=lmart_x_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])




{'map': 0.06321885365780862,
 'ndcg': 0.318820852309049,
 'ndcg_cut_30': 0.07544726686795213,
 'P_10': 0.08958333333333336}

In [ ]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.01,
      gamma=0.6,
      min_child_weight=0.20,
      max_depth=6,
      verbose=2,
      random_state=42)

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2","WMODEL:DPH",])
lmart_x_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels_val = pt.io.read_qrels(qrels_path)
val = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
lmart_x_pipe.fit(train.head(5), qrels_train,train.tail(3),qrels_train)

 
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=lmart_x_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])




{'map': 0.05592170986342046,
 'ndcg': 0.31002038604706356,
 'ndcg_cut_30': 0.0656763532715519,
 'P_10': 0.09375000000000001}

In [ ]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.001,
      gamma=0.6,
      min_child_weight=0.20,
      max_depth=6,
      verbose=2,
      random_state=42)

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2","WMODEL:DPH",])
lmart_x_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels_val = pt.io.read_qrels(qrels_path)
val = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
lmart_x_pipe.fit(train.head(5), qrels_train,train.tail(3),qrels_train)

 
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=lmart_x_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])




{'map': 0.05338719593066205,
 'ndcg': 0.3078358033386011,
 'ndcg_cut_30': 0.06733731734795811,
 'P_10': 0.09166666666666669}

In [ ]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.001,
      gamma=0.6,
      min_child_weight=0.20,
      max_depth=3,
      verbose=2,
      random_state=42)

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2","WMODEL:DPH",])
lmart_x_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels_val = pt.io.read_qrels(qrels_path)
val = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
lmart_x_pipe.fit(train.head(5), qrels_train,train.tail(3),qrels_train)

 
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=lmart_x_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])




{'map': 0.049991376278375126,
 'ndcg': 0.301591280378368,
 'ndcg_cut_30': 0.05485946441410209,
 'P_10': 0.06875000000000002}

In [ ]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.001,
      gamma=0.6,
      min_child_weight=0.20,
      max_depth=8,
      verbose=2,
      random_state=42)

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2","WMODEL:DPH",])
lmart_x_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels_val = pt.io.read_qrels(qrels_path)
val = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
lmart_x_pipe.fit(train.head(5), qrels_train,train.tail(3),qrels_train)

 
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=lmart_x_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])




{'map': 0.061001456497800526,
 'ndcg': 0.3152496542934874,
 'ndcg_cut_30': 0.07719940366407037,
 'P_10': 0.10000000000000002}

In [ ]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.001,
      gamma=0.6,
      min_child_weight=0.20,
      max_depth=8,
      verbose=2,
      random_state=42)

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:PL2","WMODEL:DPH",])
lmart_x_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels_val = pt.io.read_qrels(qrels_path)
val = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
lmart_x_pipe.fit(train.head(5), qrels_train,train.tail(3),qrels_train)

 
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=lmart_x_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])




{'map': 0.04898003352071977,
 'ndcg': 0.3023045493887995,
 'ndcg_cut_30': 0.06118064596087822,
 'P_10': 0.075}

In [ ]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.001,
      gamma=0.6,
      min_child_weight=0.20,
      max_depth=8,
      verbose=2,
      random_state=42)

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2",])
lmart_x_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels_val = pt.io.read_qrels(qrels_path)
val = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
lmart_x_pipe.fit(train.head(5), qrels_train,train.tail(3),qrels_train)

 
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=lmart_x_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])




{'map': 0.04267305861380521,
 'ndcg': 0.2901831073647649,
 'ndcg_cut_30': 0.03737537800102259,
 'P_10': 0.05625}

In [ ]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.001,
      gamma=0.6,
      min_child_weight=0.20,
      max_depth=8,
      verbose=2,
      random_state=42)

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf","WMODEL:DPH",])
lmart_x_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels_val = pt.io.read_qrels(qrels_path)
val = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
lmart_x_pipe.fit(train.head(5), qrels_train,train.tail(3),qrels_train)

 
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=lmart_x_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])




{'map': 0.0590667179686341,
 'ndcg': 0.3151132466076479,
 'ndcg_cut_30': 0.06577177472318797,
 'P_10': 0.08750000000000002}

In [ ]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.001,
      gamma=0.6,
      min_child_weight=0.20,
      max_depth=8,
      verbose=2,
      random_state=42)

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf",])
lmart_x_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels_val = pt.io.read_qrels(qrels_path)
val = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
lmart_x_pipe.fit(train.head(5), qrels_train,train.tail(3),qrels_train)

 
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=lmart_x_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])




{'map': 0.054576508995328815,
 'ndcg': 0.3071230149298111,
 'ndcg_cut_30': 0.049407057730656206,
 'P_10': 0.05625}

Learning to Rank Coordinate Ascent (FastRank)

In [ ]:
!pip install fastrank
import fastrank
train_request = fastrank.TrainRequest.coordinate_ascent()
params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2","WMODEL:DPH",])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))


ca_pipe = pipeline >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train, qrels_train)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=ca_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])


{'map': 0.15585538538547464,
 'ndcg': 0.4137637814474775,
 'ndcg_cut_30': 0.1711341335950635,
 'P_10': 0.19583333333333328}

In [ ]:
train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=[ "WMODEL:PL2","WMODEL:DPH",])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))


ca_pipe = pipeline >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train, qrels_train)
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=ca_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])


{'map': 0.13667064564602752,
 'ndcg': 0.4008413478101755,
 'ndcg_cut_30': 0.15567381883818007,
 'P_10': 0.17916666666666659}

In [ ]:
train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf" ,"WMODEL:PL2",])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))


ca_pipe = pipeline >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train, qrels_train)
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=ca_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])


{'map': 0.15236851513958252,
 'ndcg': 0.41147813117901516,
 'ndcg_cut_30': 0.17265562073954419,
 'P_10': 0.19999999999999996}

In [ ]:
train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf" ,"WMODEL:DPH",])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))


ca_pipe = pipeline >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train, qrels_train)
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=ca_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])


{'map': 0.12381557379601928,
 'ndcg': 0.3868334878479911,
 'ndcg_cut_30': 0.12885418554506328,
 'P_10': 0.1333333333333333}

In [ ]:
train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf" ,"WMODEL:DPH",])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))


ca_pipe = pipeline >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train, qrels_train)
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=ca_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])


In [ ]:
train = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels_train = pt.io.read_qrels(qrels_path)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf"])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x.split(':')[-1].split('.')[0])
qrels_train['docno']= qrels_train['docno'].apply(lambda x: x +'-'+str(int(x.split('_')[1])+120))


ca_pipe = pipeline >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train, qrels_train)
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")
res=ca_pipe.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])


{'map': 0.06802270383160423,
 'ndcg': 0.32450997566562295,
 'ndcg_cut_30': 0.06328095754272588,
 'P_10': 0.07916666666666668}